In [1]:
import sources.logins as lg
from time import sleep
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

warnings.filterwarnings('ignore')

def login_exfm(driver_on = True):
    # generate key using fernet
    # key = Fernet.generate_key().decode()
    key = 'cLAiAdZU1U0sxWWK8sxhF0IWIBP4KrR3xhdu3x1EDQI='
    
    
    # login ExFM and download latest incompleted data
    name,pw = lg.save_id('exfm.txt',key)
    driver = lg.login_url('exfm',name,pw)
    
    
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search	
            break
        except Exception as e:
            print ('Wrong Username or Password. Try again...')
            name,pw = lg.save_id('exfm.txt',key)
            driver = lg.login_url('exfm',name,pw)
    
    d_type_menu = [
                    'Incompleted',
                    'History',
                    'Equipments',
                    'Customers',
                    'No Download'
                    
                    ]
    #border table
    print(f'\n{"_"*50}')
    print(f'{"|  No.|  Function": <49}|')
    print(f'|{"_"*48}|')
    for i in range(1,1+len(d_type_menu)):
        print(f'|{i: >3}  |  {d_type_menu[i-1]: <40}|')
    print(f'|{"_"*48}|') #bottom border
    while True:
        ind = str(input('Select Dataset to download: '))
        try:
            ind = int(ind)
        except:
            if ind.upper() == 'Q' or ind.upper() == 'QUIT':
                break
            else:
                print('Only accept number')
                continue
        if 0<ind<=len(d_type_menu):
            d_type = d_type_menu[ind-1]
            break
        else:
            print(f'Input number must be less than {len(d_type_menu)}\n')
    
    print(d_type)
    
    # driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search
    
    if d_type == d_type_menu[0]: # Incompleted
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[1]: # History
        Select(driver.find_element_by_xpath('//*[@id="sidIN_REPAIR_STATUS"]')).select_by_index(0)
        print('Select Status "All" and download.')
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[2]: # Equipments
        driver.find_element_by_xpath('//*[@id="EQP_MGT_LINK"]').click() # Equipments
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    
    if d_type == d_type_menu[3]: # Customers
        driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click() 
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    else:
        print('No Download')

    # check latest file
    i_wait = 0
    while True:
        try: # check file already
            file, ctime = lg.file_latest(folder_name='Downloads')
            if file.endswith('xls'):
                print (f'\nDonwload file {file} succesful at {ctime}')
                break
            else:
                print(f'Please wait for compressing file {file}...{i_wait}s')
                i_wait +=3
                sleep(3)
        except Exception as e: #waiting for download
            if d_type == d_type_menu[-1]: break
            print(f'Please wait for dowloading...{i_wait}s')
            i_wait +=3
            sleep(3)
            
    if driver_on:
        return driver
    else:
        #finish close driver
        driver.close()
    sleep(2)

def auto_accept(driver):
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    driver.switch_to.alert.accept()

In [2]:

driver = login_exfm()


Select account:
1   |   thongle
2   |   Nguyen
0   |   CREATE NEW ACCOUNT
Select account by number: 1
thongle is selected to login
Removed Downloads


Create new folder: Downloads
Select Chrome Driver and press enter for authorize Certificate...

Typing your ID...
Typing your password...
Login sucessful!


__________________________________________________
|  No.|  Function                                |
|________________________________________________|
|  1  |  Incompleted                             |
|  2  |  History                                 |
|  3  |  Equipments                              |
|  4  |  Customers                               |
|  5  |  No Download                             |
|________________________________________________|
Select Dataset to download: 5
No Download
No Download


In [ ]:
type(driver)

In [3]:
from sqlite3 import connect
import pandas as pd
conn = connect('history.db')
# file_name = 'files/ACC_TBL_EXPORT_230912.xlsx'
# dealers = pd.read_excel(file_name)
# dealers.to_sql('dealers',conn,index=False,if_exists='replace')

In [6]:
sn_list = '''
7G391K201
7G391K201

'''
sn_list = tuple(sn_list.strip().split('\n'))

In [8]:
q = f'''
        SELECT [rma no.],customer_name,repair_status,[status info],
        FROM consolidated c
        WHERE serial_no in {sn_list}
        ORDER BY [rma no.] DESC
    '''
pd.read_sql(q,conn)

,RMA No.,CUSTOMER_NAME,REPAIR_STATUS,Status Info
0,FMSV2023040010,Hanoi Medical University Hospital,Completed,Awating Contract billing
1,FMSV2021120067,Hanoi Medical University Hospital,Completed,None
2,FMSV2020120030,Hanoi Medical University Hospital,Completed,None


In [ ]:
import sources.inspection as ins
a = ins.Auto_QC()

In [ ]:
# sn = '7G391K201'
for sn in sn_list:
    a.return_unrepair(driver,sn,approval_index='Transfer')

In [ ]:
q = '''
        SELECT * 
        
        FROM dealers
        WHERE territory1 = 'ETC'
    '''
north = pd.read_sql(q,conn)
north

In [ ]:
from time import sleep

In [ ]:
cus_no_done=[]

In [ ]:
#click customer tab
driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click()
for cus_no in north['No.']:
    if cus_no in cus_no_done:
        print(f' \nPassed {cus_no}')
    else:
        customer = dealers[dealers['No.']==cus_no]
        customer = customer.reset_index()
    #     display(customer)
        #search customer no.
        driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').clear()
        driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').send_keys(cus_no)

        #click search Button
        driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()

        # select first row
        driver.find_element_by_xpath('//*[@id="g[0].sidDISP_CODE_field"]').click()
        sleep(0.5)

        # edit by file
        name = driver.find_element_by_xpath('//*[@id="sidNAME"]')
        name.clear()
        name.send_keys(customer['Pro Name'])
        print(cus_no,customer['Pro Name'])
        territory = driver.find_element_by_xpath('//*[@id="sidTEXT_TERRITORY_NAME"]')
        territory.clear()
        try:
            territory.send_keys(customer['Territory1'])
        except Exception as e:
            print(e)
        area = driver.find_element_by_xpath('//*[@id="sidTEXT_AREA_NAME"]')
        area.clear()
        try:
            area.send_keys(customer['Area1'])
        except Exception as e:
            print(e)

        city = driver.find_element_by_xpath('//*[@id="sidCITY"]')
        city.clear()
        try:
            city.send_keys(customer['City1'])
        except Exception as e:
             print(e)
        
        #clear email
        driver.find_element_by_xpath('//*[@id="sidCONTACT1_PERSON_MAIL"]').clear()
        driver.find_element_by_xpath('//*[@id="sidCONTACT2_PERSON_MAIL"]').clear()
        driver.find_element_by_xpath('//*[@id="sidCONTACT3_PERSON_MAIL"]').clear()
        
        
        # click save
        driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
        sleep(0.5)
        #click return
        driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
        
        sleep(0.5)
        cus_no_done.append(cus_no)
        
        
print('Done')

In [ ]:
cus_no_done

In [ ]:
# manual check
cus_no = 'FMSV00130'
customer = dealers[dealers['No.']==cus_no]
customer = customer.reset_index()

In [ ]:
#click customer tab
driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click()

#search customer no.
cus_no = 'FMSV00189'#customer['No.'][0]
driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').clear()
driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').send_keys(cus_no)

#click search Button
driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()

# select first row
driver.find_element_by_xpath('//*[@id="g[0].sidDISP_CODE_field"]').click()

In [ ]:
# edit by file
name = driver.find_element_by_xpath('//*[@id="sidNAME"]')
name.clear()
name.send_keys(customer['Pro Name'])

territory = driver.find_element_by_xpath('//*[@id="sidTEXT_TERRITORY_NAME"]')
territory.clear()
try:
    territory.send_keys(customer['Territory1'])
except Exception as e:
    print(e)
area = driver.find_element_by_xpath('//*[@id="sidTEXT_AREA_NAME"]')
area.clear()
try:
    area.send_keys(customer['Area1'])
except Exception as e:
    print(e)

city = driver.find_element_by_xpath('//*[@id="sidCITY"]')
city.clear()
try:
    city.send_keys(customer['City1'])
except Exception as e:
     print(e)

In [ ]:
# click save
driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
sleep(0.5)
#click return
driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
sleep(0.5)

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()